In [74]:
import requests
import json
from datetime import datetime
import pprint
import pandas as pd

In [143]:
address = input("Input your current location, if possible enter your exact address: ")

def geocode(location):
    res = requests.get(f"https://geocode.xyz/{location}", params={"json":1})
    data = res.json()
    return {
        "type":"Point",
        "coordinates":[float(data["longt"]),float(data["latt"])]
    }

def getCoords(location):
    my_coordinates = geocode(location)
    return my_coordinates["coordinates"]

Input your current location, if possible enter your exact address: Somo, Santander


In [144]:
def getBeach(lat,longt):

    """
        Function that returns the closest beach to the location
        inputted by the user
    """

    url = f"https://surfvideos.xyz/locations.json?latitude={lat}&longitude={longt}&distance=50"

    payload = {}
    headers = {
    'X-User-Email': 'ap1468@georgetown.edu',
    'X-User-Token': 'edJsbzQC2YbM2bYFPWYm',
    'Accept': '{{Accept}}',
    'Content-Type': '{{Content}}'
    }

    response = requests.request("GET", url, headers=headers, data = payload)
    data = response.json()

    for i in range(len(data)):
        lat = data[i]["data"]["attributes"]["latitude"]
        longt = data[i]["data"]["attributes"]["longitude"]
        
        return f"{lat},{longt}"


In [145]:
def getMoreBeaches(lat, longt):
    url = f"https://surfvideos.xyz/locations.json?latitude={lat}&longitude={longt}&distance=50"

    payload = {}
    headers = {
    'X-User-Email': 'ap1468@georgetown.edu',
    'X-User-Token': 'edJsbzQC2YbM2bYFPWYm',
    'Accept': '{{Accept}}',
    'Content-Type': '{{Content}}'
    }

    response = requests.request("GET", url, headers=headers, data = payload)
    data = response.json()
    
    df = pd.DataFrame(
    [
        data[i]["data"]["attributes"]["name"], 
        data[i]["data"]["attributes"]["address"], 
        data[i]["data"]["attributes"]["direction"], 
        data[i]["data"]["attributes"]["experience"], 
        data[i]["data"]["attributes"]["bottom"], 
        data[i]["data"]["attributes"]["best_wind_direction"], 
        data[i]["data"]["attributes"]["best_swell_direction"],
        data[i]["data"]["attributes"]["latitude"],
        data[i]["data"]["attributes"]["longitude"]
] for i in range(len(data)))
    
    df = df.rename(columns={0: 'Name', 1: 'Address', 2: 'Direction', 3: 'Experience',4: 'Bottom', 5: 'Best wind', 6: 'Best swell', 7: "Latitude", 8:"Longitude"})
    df["latlong"] = None
  
    for i in range(df.shape[0]):
        df["latlong"][i] = f"{df['Latitude'][i]},{df['Longitude'][i]}"    
        
    df = df.drop(columns=['Latitude', 'Longitude'])
        
    return df

    

df = getMoreBeaches("43.479472","-3.80868")

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [146]:
latilongi = getBeach(latitude,longitude)

In [147]:

def getBeachForecast(latlong):
    key = "4bf49534c534477aa8a80201202707"
    url = f"http://api.worldweatheronline.com/premium/v1/marine.ashx?key={key}&format=json&q={latlong}"

    res = requests.get(url)
    data = res.json()

    now = datetime.now()
    current_time = now.strftime("%H" + "00")
    current_time = int(current_time)

    def getTime(current_time):
        for i in (data["data"]["weather"][0]["hourly"]):
            if current_time <= int(i["time"]):
                return(i["time"])


    for i in data["data"]["weather"][0]["hourly"]:
        if i["time"] == getTime(current_time):
            return pprint.pprint(i)

getBeachForecast(latilongi)

{'DewPointC': '23',
 'DewPointF': '74',
 'FeelsLikeC': '31',
 'FeelsLikeF': '89',
 'HeatIndexC': '31',
 'HeatIndexF': '89',
 'WindChillC': '28',
 'WindChillF': '82',
 'WindGustKmph': '36',
 'WindGustMiles': '23',
 'cloudcover': '0',
 'humidity': '77',
 'precipInches': '0.0',
 'precipMM': '0.0',
 'pressure': '1012',
 'pressureInches': '30',
 'sigHeight_m': '0.8',
 'swellDir': '80',
 'swellDir16Point': 'E',
 'swellHeight_ft': '1.6',
 'swellHeight_m': '0.5',
 'swellPeriod_secs': '5.0',
 'tempC': '28',
 'tempF': '82',
 'time': '1800',
 'visibility': '10',
 'visibilityMiles': '6',
 'waterTemp_C': '27',
 'waterTemp_F': '80',
 'weatherCode': '113',
 'weatherDesc': [{'value': 'Sunny'}],
 'weatherIconUrl': [{'value': 'http://cdn.worldweatheronline.com/images/wsymbols01_png_64/wsymbol_0001_sunny.png'}],
 'winddir16Point': 'E',
 'winddirDegree': '83',
 'windspeedKmph': '18',
 'windspeedMiles': '11'}


In [148]:
def getSwellHeight(latlong):
    key = "4bf49534c534477aa8a80201202707"
    url = f"http://api.worldweatheronline.com/premium/v1/marine.ashx?key={key}&format=json&q={latlong}"

    res = requests.get(url)
    data = res.json()

    now = datetime.now()
    current_time = now.strftime("%H" + "00")
    current_time = int(current_time)

    def getTime(current_time):
        for i in (data["data"]["weather"][0]["hourly"]):
            if current_time <= int(i["time"]):
                return(i["time"])


    for i in data["data"]["weather"][0]["hourly"]:
        if i["time"] == getTime(current_time):
            return i["swellHeight_m"]

    
df['Swell Height'] = df['latlong'].apply(getSwellHeight)

In [149]:
def getWindDir(latlong):
    key = "4bf49534c534477aa8a80201202707"
    url = f"http://api.worldweatheronline.com/premium/v1/marine.ashx?key={key}&format=json&q={latlong}"

    res = requests.get(url)
    data = res.json()

    now = datetime.now()
    current_time = now.strftime("%H" + "00")
    current_time = int(current_time)

    def getTime(current_time):
        for i in (data["data"]["weather"][0]["hourly"]):
            if current_time <= int(i["time"]):
                return(i["time"])


    for i in data["data"]["weather"][0]["hourly"]:
        if i["time"] == getTime(current_time):
            return i["winddir16Point"]

df['Wind direction'] = df['latlong'].apply(getWindDir)

In [150]:
def getSwellDir(latlong):
    key = "4bf49534c534477aa8a80201202707"
    url = f"http://api.worldweatheronline.com/premium/v1/marine.ashx?key={key}&format=json&q={latlong}"

    res = requests.get(url)
    data = res.json()

    now = datetime.now()
    current_time = now.strftime("%H" + "00")
    current_time = int(current_time)

    def getTime(current_time):
        for i in (data["data"]["weather"][0]["hourly"]):
            if current_time <= int(i["time"]):
                return(i["time"])


    for i in data["data"]["weather"][0]["hourly"]:
        if i["time"] == getTime(current_time):
            return i["swellDir16Point"]

df['Swell direction'] = df['latlong'].apply(getSwellDir)

In [153]:
def getWindSpeed(latlong):
    key = "4bf49534c534477aa8a80201202707"
    url = f"http://api.worldweatheronline.com/premium/v1/marine.ashx?key={key}&format=json&q={latlong}"

    res = requests.get(url)
    data = res.json()

    now = datetime.now()
    current_time = now.strftime("%H" + "00")
    current_time = int(current_time)

    def getTime(current_time):
        for i in (data["data"]["weather"][0]["hourly"]):
            if current_time <= int(i["time"]):
                return(i["time"])


    for i in data["data"]["weather"][0]["hourly"]:
        if i["time"] == getTime(current_time):
            return i["windspeedKmph"]

df['Wind speed'] = df['latlong'].apply(getWindSpeed)

In [ ]:
def getSwellPeriod(latlong):
    key = "4bf49534c534477aa8a80201202707"
    url = f"http://api.worldweatheronline.com/premium/v1/marine.ashx?key={key}&format=json&q={latlong}"

    res = requests.get(url)
    data = res.json()

    now = datetime.now()
    current_time = now.strftime("%H" + "00")
    current_time = int(current_time)

    def getTime(current_time):
        for i in (data["data"]["weather"][0]["hourly"]):
            if current_time <= int(i["time"]):
                return(i["time"])


    for i in data["data"]["weather"][0]["hourly"]:
        if i["time"] == getTime(current_time):
            return i["swellPeriod_secs"]

df['Swell period'] = df['latlong'].apply(getSwellPeriod)

In [ ]:
def getWaterTemp(latlong):
    key = "4bf49534c534477aa8a80201202707"
    url = f"http://api.worldweatheronline.com/premium/v1/marine.ashx?key={key}&format=json&q={latlong}"

    res = requests.get(url)
    data = res.json()

    now = datetime.now()
    current_time = now.strftime("%H" + "00")
    current_time = int(current_time)

    def getTime(current_time):
        for i in (data["data"]["weather"][0]["hourly"]):
            if current_time <= int(i["time"]):
                return(i["time"])


    for i in data["data"]["weather"][0]["hourly"]:
        if i["time"] == getTime(current_time):
            return i["waterTemp_C"]

df['Water temperature'] = df['latlong'].apply(getWaterTemp)

In [ ]:
def getWeatherDescription(latlong):
    key = "4bf49534c534477aa8a80201202707"
    url = f"http://api.worldweatheronline.com/premium/v1/marine.ashx?key={key}&format=json&q={latlong}"

    res = requests.get(url)
    data = res.json()

    now = datetime.now()
    current_time = now.strftime("%H" + "00")
    current_time = int(current_time)

    def getTime(current_time):
        for i in (data["data"]["weather"][0]["hourly"]):
            if current_time <= int(i["time"]):
                return(i["time"])


    for i in data["data"]["weather"][0]["hourly"]:
        if i["time"] == getTime(current_time):
            return i["weatherDesc"][0]["value"]
        


df['Weather Description'] = df['latlong'].apply(getWeatherDescription)

In [152]:
df

,Name,Address,Direction,Experience,Bottom,Best wind,Best swell,latlong,Swell Height,Wind direction,Swell direction
0,El Muro,"Segunda Playa del Sardinero, Avenida del Stadi...",Left,All surfers,Point-break,"[West, SouthWest, South]","[North, NorthWest, West]","43.479472,-3.788223",1.2,NNE,NNW
1,Matalenas,"Playa de Mataleñas, Acceso a la playa de Matal...",Right and left,All surfers,Beach-break,"[NorthWest, West, SouthWest]","[North, NorthWest]","43.485851,-3.788137",1.2,NNE,NNW
2,El sardinero,"Segunda Playa del Sardinero, Avenida del Stadi...",None,Experienced surfers,Beach-break,"[SouthWest, South]","[North, NorthWest]","43.475564,-3.783889",1.2,NNE,NNW
3,Somo,"Playa de El Puntal, Kostaldeko Donejakue bidea...",Right and left,All surfers,Beach-break,"[SouthWest, South, SouthEast, East]","[North, NorthWest]","43.457031,-3.744407",1.2,N,NNW
4,Isla Santa Marina,"Playa de los Tranquilos, Kostaldeko Donejakue ...",Right,Pros or kamikaze only...,Reef-rocky,"[SouthEast, East, NorthEast]",[NorthWest],"43.470148,-3.731216",1.2,N,NNW
5,Covachos,"Tómbolo de Covachos, Barrio La Arnía, Santa Cr...",Right,All surfers,Reef-rocky,None,None,"43.473726,-3.909309",1.2,NNE,NNW
6,De langre,"Playa de Langre, Kostaldeko Donejakue bidea, R...",None,Experienced surfers,Sand-bar,[South],[West],"43.475704,-3.689454",1.2,N,NNW
7,Galizano Point,"Kostaldeko Donejakue bidea, Ribamontán al Mar,...",Right,Experienced surfers,Reef-rocky,"[SouthWest, South]","[North, NorthWest]","43.480344,-3.67983",1.2,N,NNW
8,Galizano,"Playa de Galizano, Acceso a Playa de San Migue...",Right,All surfers,Reef-rocky,"[South, SouthEast, East]",[NorthWest],"43.480211,-3.675646",1.2,N,NNW
9,Liencres,"Playa de Canallave, Acceso Playa de Valdearena...",Right and left,All surfers,Beach-break,"[SouthWest, South, SouthEast, East]","[North, NorthWest]","43.453604,-3.962116",1.2,NNE,NNW


In [69]:
location = input("Input your current location, if possible enter your exact address: ")

latlong = getCoords(location)
longitude = latlong[0]
latitude = latlong[1]

latitude_longitude = getBeach(latitude,longitude)
getBeachForecast(latitude_longitude)

# Santander, Cantabria, Spain

Input your current location, if possible enter your exact address: Ibiza, Spain
{'DewPointC': '22',
 'DewPointF': '72',
 'FeelsLikeC': '31',
 'FeelsLikeF': '88',
 'HeatIndexC': '31',
 'HeatIndexF': '88',
 'WindChillC': '28',
 'WindChillF': '82',
 'WindGustKmph': '32',
 'WindGustMiles': '20',
 'cloudcover': '10',
 'humidity': '71',
 'precipInches': '0.0',
 'precipMM': '0.0',
 'pressure': '1014',
 'pressureInches': '30',
 'sigHeight_m': '0.6',
 'swellDir': '80',
 'swellDir16Point': 'E',
 'swellHeight_ft': '1.3',
 'swellHeight_m': '0.4',
 'swellPeriod_secs': '4.5',
 'tempC': '28',
 'tempF': '82',
 'time': '1800',
 'visibility': '10',
 'visibilityMiles': '6',
 'waterTemp_C': '27',
 'waterTemp_F': '80',
 'weatherCode': '116',
 'weatherDesc': [{'value': 'Partly cloudy'}],
 'weatherIconUrl': [{'value': 'http://cdn.worldweatheronline.com/images/wsymbols01_png_64/wsymbol_0002_sunny_intervals.png'}],
 'winddir16Point': 'SE',
 'winddirDegree': '126',
 'windspeedKmph': '19',
 'windspeedMiles': '12